In [0]:
%pip install -r requirements.txt
%restart_python

In [0]:
from sktime.split import temporal_train_test_split
from sktime.forecasting.base import ForecastingHorizon
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error, mean_squared_error
from sktime.forecasting.residual_booster import ResidualBoostingForecaster
from sklearn.ensemble import GradientBoostingRegressor
from sktime.forecasting.compose import make_reduction
from sktime.forecasting.ets import AutoETS
from sktime.forecasting.model_evaluation import evaluate
from sktime.forecasting.model_selection import ExpandingWindowSplitter
from sktime.forecasting.compose import TransformedTargetForecaster
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mlflow
import shap
import json
import re
import seaborn as sns
from copy import deepcopy
from itertools import product
from pathlib import Path

## Model Development for Total Household Deposits

This notebook documents the model development process for forecasting total household deposits. The forecasting pipeline combines an **AutoETS base forecaster** with a **residual boosting model** using **Gradient Boosting Regression**.

The **AutoETS model** is a time series forecaster that automatically selects the best configuration of exponential smoothing (ETS) components — error, trend, and seasonality. It captures the primary linear, seasonal, and trend-driven structure of the time series. 

**Note: AutoETS is used instead of manually specifying ETS components as it automatically selects the best combination of Trend, Seasonality and Error using AICc. It provides a robust, consistent and interpretable baseline across time so that the residual booster can focus purely on what ETS can't explain and is relatively immune to overfitting*

Given that household deposits exhibit strong linear, non-stationary and seasonal patterns, AutoETS serves as the ideal base model for capturing these dominant patterns. 
However, to enhance the forecast and capture **nonlinear relationships** or **exogenous effects** not explained by the base model, a **Gradient Boosting model is trained on the residuals** (errors) from the AutoETS forecast.

This two-stage modeling approach is designed to combine the strengths of statistical forecasting and machine learning.  
The development process consists of two key stages:  
- **SHAP-Based Feature selection** using average SHAP importance across cross-validation folds, applied to the residual boosting model.  
- **Hyperparameter tuning** of the Gradient Boosting regressor to optimize residual learning performance.


In [0]:
def create_features(df, target_col, predictor_cols, lags=[1,3,4, 6,12], rolling_windows=[3,4, 6,12]):
    """
    Creates lag-based and rolling statistical features for both the target and predictor columns.

    Parameters:
    - df (pd.DataFrame): Input DataFrame with 'date', target, and predictor columns
    - target_col (str): Name of the target column
    - predictor_cols (list): List of predictor column names
    - lags (list): Lags to apply for lag, diff, and pct change features
    - rolling_windows (list): Window sizes for rolling statistics (mean, std, min, max)

    Returns:
    - pd.DataFrame: DataFrame with engineered features and datetime features (year, month, quarter)
    """
    # Ensure date is in datetime format
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date')
    df.set_index('date', inplace=True)

    new_features = {}

    # For each predictor (and target), compute lags, rolling stats, etc.
    for col in [target_col] + predictor_cols:
        # Lag features
        for lag in lags:
            new_features[f'{col}_lag{lag}'] = df[col].shift(lag) # lag

        # Differences and pct changes
        for lag in lags:
            new_features[f'{col}_diff{lag}'] = df[col].diff(lag) # difference
            if (df[col] != 0).all():
                new_features[f'{col}_roc{lag}'] = df[col].pct_change(lag) # rate of change

        # Rolling stats
        for win in rolling_windows:
            new_features[f'{col}_ma{win}'] = df[col].rolling(win).mean()
            new_features[f'{col}_std{win}'] = df[col].rolling(win).std()
            new_features[f'{col}_min{win}'] = df[col].rolling(win).min()
            new_features[f'{col}_max{win}'] = df[col].rolling(win).max()

    # Combine original and new features        
    df_new = pd.concat([df]+ [pd.DataFrame(new_features, index=df.index)], axis=1)

    # Add datetime features
    df_new['year'] = df_new.index.year
    df_new['month'] = df_new.index.month
    df_new['quarter'] = df_new.index.quarter

    # Drop rows with missing values caused by shifting/rolling
    df_new = df_new.dropna()
    df_new.reset_index(inplace=True)

    return df_new

In [0]:
## Read Data
raw_df = pd.read_csv('../data/processed/combined_total_household_data_interpolate.csv')
raw_df = raw_df.sort_values('date')

# all significant features retained after the feature reduction phase
significant_cols = ['card_consumables', 'card_credit','household_loans', 'google_home_loan_rate_search', 'unemployment_rate','employed_labour_force', 'labour_force_participation_rate','labour_cost_index', 'cpi', 'cpi_housing_household', 'cpi_qq', 'cpi_yy',
'production_based_gdp', 'production_based_gdp_qq','production_based_gdp_yy']
cpi_features = ['cpi', 'cpi_housing_household', 'cpi_qq', 'cpi_yy'] # cpi realted features, only one should be selected if any are significant at all
gdp_features = ['production_based_gdp', 'production_based_gdp_qq','production_based_gdp_yy'] # gdp realted features, only one should be selected if any are significant at all

raw_df.head(5)

In [0]:
def plot_expanding_cv_splits(y, initial_window, step_length, fh):
    """
    Plot expanding window CV with dates on the x-axis to experiment and visualize the CV train test splits.
    
    Parameters:
        y (pd.Series): time series with PeriodIndex or DatetimeIndex
        initial_window (int): initial training window size
        step_length (int): step size between folds
        fh (int): forecast horizon (e.g. 1, 2, etc.)
    """
    if isinstance(y.index, pd.PeriodIndex):
        x_dates = y.index.to_timestamp()
    else:
        x_dates = y.index

    cv = ExpandingWindowSplitter(
        initial_window=initial_window,
        step_length=step_length,
        fh=[fh]
    )

    plt.figure(figsize=(15, 6))
    n_splits = cv.get_n_splits(y)

    test_dates = []
    for i, (train_idx, test_idx) in enumerate(cv.split(y)):
        plt.plot(x_dates[train_idx], [i] * len(train_idx), 'b.', label='Train' if i == 0 else "")
        plt.plot(x_dates[test_idx], [i] * len(test_idx), 'r.', label='Test' if i == 0 else "")
        test_dates.append(x_dates[test_idx][0])

    formatted_test_dates = [d.strftime('%Y-%m') for d in test_dates]
    print("All test dates:", formatted_test_dates)
    
    plt.title(f"Expanding Window CV — {n_splits} folds | Init={initial_window}, Step={step_length}, FH={fh}")
    plt.xlabel("Date")
    plt.ylabel("Fold")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


# Target setup
df_train, df_test = temporal_train_test_split(raw_df, test_size=6)
df = df_train.set_index('date').asfreq('MS')
df.index = df.index.to_period("M")
y = df['household_deposits']

# Inital Parameters
n_folds = 10
horizon = 14
N = len(y)
step_length =  6

# Back-calculate initial window so that last fold is recent
initial_window = N - ((n_folds - 1) * step_length + horizon)

print(f"Initial window: {initial_window}, Step length: {step_length}")

# Plot
plot_expanding_cv_splits(y, initial_window=initial_window, step_length=step_length, fh=horizon)


## SHAP-Based Feature Selection Process

To select the most predictive features for the residual component of the AutoETS + Gradient Boosting pipeline, a cross-validated SHAP-based feature selection procedure was implemented. The gradient booster for residual boosting is trained to learn and correct the residuals (forecast errors) produced by the AutoETS model.

For each fold in an expanding window cross-validation:

- A base AutoETS model is fit on a sub-window to forecast household deposits *h* months ahead.
- Within each fold, a series of forecasts is made using expanding sub-windows, generating residuals between AutoETS predictions and actual values at horizon *h*.
- These residuals are treated as the target for a Gradient Boosting Regressor (GBR), which is trained on exogenous features including lagged and rolling statistics.
- SHAP values are computed for each trained GBR to assess feature contributions to residual prediction.
- Absolute SHAP values are collected for all folds and averaged to produce stable, cross-validated feature importance rankings.

At the end of the process, *k* residual models (one per fold) yield *k* sets of SHAP importances. These are averaged to determine the most consistently important features across time, and the top-ranked variables are selected for use in each horizon-specific forecast model. This method ensures that selected features are not only predictive but also temporally robust, capturing consistent patterns in forecast errors across time.

In [0]:
# use shap
def average_shap_feature_selector_cv(df,predictor_cols,lags=[1, 3, 4],rolling_windows=[3, 4],h=2,n_folds=5, step_length=10,sp=12,n_estimators=100, learning_rate=0.1,max_depth=3, initial_training_window = 80):
    """
    Performs cross-validated SHAP-based feature importance estimation using residual boosting.

    This function fits an AutoETS model to forecast `household_deposits`, computes residuals
    at each horizon `h`, and fits a Gradient Boosting Regressor to model these residuals. 
    SHAP values are calculated on the boosting model to estimate feature importance across CV folds.

    Parameters:
    ----------
    df : pd.DataFrame
        Input dataframe with a 'date' column, target 'household_deposits', and predictors.
    predictor_cols : list
        List of predictor variable names to engineer features from.
    lags : list
        List of lag values for feature engineering.
    rolling_windows : list
        Rolling window sizes for statistical feature creation.
    h : int
        Forecast horizon (e.g., 2 = forecast 2 months ahead).
    n_folds : int
        Number of cross-validation folds.
    step_length : int
        Step between CV fold start dates.
    sp : int
        Seasonal period for AutoETS (set to 12 for the deposits data as strong yearly seasonaility observed).
    n_estimators : int
        Number of trees for GradientBoostingRegressor.
    learning_rate : float
        Learning rate for the gradient boosting model.
    max_depth : int
        Maximum depth of trees in GBT model.
    initial_training_window : int
        Minimum number of points for inner expanding window (ETS model training).

    Returns:
    --------
    feature_importance : pd.DataFrame
        DataFrame with 'feature', 'mean_abs_shap', and 'std_abs_shap', sorted by importance.
    """

    # Prepare Data
    df = df[['date', 'household_deposits'] + predictor_cols].copy()
    df = create_features(df, 'household_deposits', predictor_cols, lags=lags, rolling_windows=rolling_windows)
    df = df.set_index('date').asfreq('MS')
    df.index = df.index.to_period("M")
    X = df.drop(columns='household_deposits')
    y = df['household_deposits']
    N = len(X)

    # Define initial window for outer cross-validation
    initial_window = N - ((n_folds - 1) * step_length + h)

    splitter = ExpandingWindowSplitter(
        initial_window=initial_window,
        step_length=step_length,
        fh=[h]
    )

    shap_list = []
    feature_names = X.columns

    # Cross-validated Residual Modeling
    for fold_i, (train_idx, test_idx) in enumerate(splitter.split(y)):
        print(f"Fold {fold_i+1}/{n_folds}")

        residuals_list = []
        features_list = []
        print(f'start: {initial_training_window} --> end: {train_idx[-h]}')
        # Inner expanding window loop to simulate walk-forward residual estimation
        for j in range(initial_training_window, train_idx[-h] + 1):  # expand from initial window to fold train end
            y_train_sub = y.iloc[:j]
            X_train_sub = X.iloc[:j]

            # Train ETS on sub-window
            ets = AutoETS(auto=True, sp=sp, n_jobs=-1)
            ets.fit(y_train_sub)

            # Predict h steps ahead
            y_pred = ets.predict(fh=[h])

            # Actual value at forecast date
            forecast_timestamp = y_pred.index[0]
            actual_val = y.loc[forecast_timestamp]  

            # Residual at horizon h
            residual = actual_val - y_pred.iloc[0]

            # Feature vector aligned with actual_val date
            X_feat = X.loc[forecast_timestamp]

            # Collect the residuals at every time point
            residuals_list.append(residual)
            features_list.append(X_feat)


        # Train gradient boosting on residuals
        residuals_fold = pd.Series(residuals_list)
        X_resid_fold = pd.DataFrame(features_list)

        reg = GradientBoostingRegressor(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            random_state=42
        )
        reg.fit(X_resid_fold, residuals_fold)

        # Compute SHAP values
        explainer = shap.TreeExplainer(reg)
        shap_vals = explainer.shap_values(X_resid_fold)

        # Store abs SHAP values for this fold
        shap_list.append(np.abs(shap_vals))

    # Aggregate SHAP over folds
    shap_matrix = np.vstack(shap_list)
    mean_abs_shap = shap_matrix.mean(axis=0)
    std_abs_shap = shap_matrix.std(axis=0)

    # Construct feature importance DataFrame
    feature_importance = pd.DataFrame({
        'feature': feature_names,
        'mean_abs_shap': mean_abs_shap,
        'std_abs_shap': std_abs_shap
    }).sort_values(by='mean_abs_shap', ascending=False)

    print("Average SHAP feature importance across folds:")
    print(feature_importance.head(20))

    return feature_importance


In [0]:
# SHAP-Based Feature Selection

# Initialise inputs and parameters
significant_cols = [
    'card_consumables', 'card_credit', 'household_loans', 'google_home_loan_rate_search',
    'unemployment_rate', 'employed_labour_force', 'labour_force_participation_rate',
    'labour_cost_index', 'cpi', 'cpi_housing_household', 'cpi_qq', 'cpi_yy',
    'production_based_gdp', 'production_based_gdp_qq', 'production_based_gdp_yy'
]

cpi_features = ['cpi', 'cpi_housing_household', 'cpi_qq', 'cpi_yy']
gdp_features = ['production_based_gdp', 'production_based_gdp_qq', 'production_based_gdp_yy']
horizons = [2, 5, 8, 14]

# Hold out the last 6 months (small test set to maximize training for SHAP
df_train, df_test = temporal_train_test_split(raw_df, test_size=6) 
shap_dict = {}

# Run SHAP feature selection for each horizon
for h in horizons:
    print(f"\n=== Running SHAP selection for Horizon {h} ===")
    
    shap_importance = average_shap_feature_selector_cv(
        df=df_train,
        predictor_cols=significant_cols,
        lags=[1,2,3,4,5,6,7,8,9,10,11,12],              # Lags 1 to 12 - include all lags
        rolling_windows=[2,3,4,5,6,7,8,9,10,11,12],     # Rolling windows 2 to 12 - include all rolling features
        h=h,
        n_folds=10,
        step_length=6,
        sp=12,
        n_estimators=100,
        learning_rate=0.1,
        max_depth=4
    )
    
    # Save and summarize
    shap_df = pd.DataFrame(shap_importance)
    shap_df.to_csv(f'tuning_results/shap_feature_selection_results_h{h}.csv')
    mean_shap = shap_df['mean_abs_shap'].mean()
    print(f"Mean SHAP for horizon {h}: {mean_shap:.6f}")

    shap_df = shap_df[shap_df['mean_abs_shap'] > 0]
    shap_dict[h] = shap_df

In [0]:
# Utilities for SHAP visualisations
def base_feat_name(feat):
    """Extract base feature name by removing lag/rolling/stat suffixes."""
    return re.sub(r'_(lag|ma|std|min|max|diff|roc)[0-9]+$', '', feat)

def make_rank_heatmap(shap_dict, top_n=20):
    """Create heatmap showing feature ranks across horizons (based on mean SHAP)."""
    all_feats = set()
    for df in shap_dict.values():
        df['base_feature'] = df['feature'].apply(base_feat_name)
        all_feats.update(df['base_feature'].unique())

    rank_data = pd.DataFrame(index=sorted(all_feats), columns=shap_dict.keys())

    for h, df in shap_dict.items():
        df['base_feature'] = df['feature'].apply(base_feat_name)
        grouped = df.groupby('base_feature')['mean_abs_shap'].mean()
        ranked = grouped.rank(ascending=False, method='min')
        rank_data.loc[ranked.index, h] = ranked.astype('Int64')

    # Get top N features across all horizons
    min_ranks = pd.to_numeric(rank_data.min(axis=1), errors='coerce')
    top_feats = min_ranks.nsmallest(top_n).index
    rank_data = rank_data.loc[top_feats]

    plt.figure(figsize=(10, max(6, len(top_feats) * 0.3)))
    sns.heatmap(rank_data.astype(float), annot=True, cmap="YlOrRd_r", cbar_kws={'label': 'Rank (1 = most important)'})
    plt.xlabel("Forecast Horizon (months)")
    plt.ylabel("Feature")
    plt.title(f"SHAP Feature Rank Across Horizons (Top {top_n})")
    plt.tight_layout()
    plt.show()

def plot_shap_summary(shap_df, horizon, top_k=10):
    """Plot SHAP importance for top raw and base features."""

    shap_df = shap_df.copy()
    shap_df['base_feature'] = shap_df['feature'].apply(base_feat_name)

    # Top raw features
    top_feats = shap_df.sort_values('mean_abs_shap', ascending=False).head(top_k)

    # Grouped bar chart: mean and std side by side
    fig, ax = plt.subplots(figsize=(10, 6))
    x = range(len(top_feats))
    bar_width = 0.4

    ax.bar([i - bar_width/2 for i in x], top_feats['mean_abs_shap'], width=bar_width, label='Mean |SHAP|', color='skyblue')
    ax.bar([i + bar_width/2 for i in x], top_feats['std_abs_shap'], width=bar_width, label='Std |SHAP|', color='salmon')

    ax.set_xticks(x)
    ax.set_xticklabels(top_feats['feature'], rotation=45, ha='right')
    ax.set_title(f"H{horizon} - Raw Feature SHAP (Top {top_k})")
    ax.set_ylabel("|SHAP value|")
    ax.legend()
    plt.tight_layout()
    plt.show()

    # Aggregated by base feature (sum & mean)
    base_summary = shap_df.groupby('base_feature').agg(
        mean_shap=('mean_abs_shap', 'mean'),
        sum_shap=('mean_abs_shap', 'sum')
    ).sort_values('sum_shap', ascending=False)

    top_base = base_summary.head(top_k)

    fig, axs = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

    axs[0].barh(top_base.index[::-1], top_base['sum_shap'][::-1], color='steelblue')
    axs[0].set_title(f"H{horizon} - Base Features by Sum |SHAP|")
    axs[0].set_xlabel("Sum |SHAP value|")

    axs[1].barh(top_base.index[::-1], top_base['mean_shap'][::-1], color='mediumseagreen')
    axs[1].set_title(f"H{horizon} - Base Features by Mean |SHAP|")
    axs[1].set_xlabel("Mean |SHAP value|")

    plt.suptitle(f"H{horizon} - Aggregated Base Feature Importance (Top {top_k})")
    plt.tight_layout()
    plt.show()


# Visualize the Shap results
shap_dict = {}
horizons = [2, 5, 8, 14]  # Or load dynamically

for h in horizons:
    print(f"\n=== Horizon {h} ===")
    shap_df = pd.read_csv(f'tuning_results/shap_feature_selection_results_h{h}.csv')
    shap_dict[h] = shap_df
    plot_shap_summary(shap_df, horizon=h, top_k=10)

# Final heatmap across all horizons
make_rank_heatmap(shap_dict, top_n=20)



**Feature Selection for each Horizon**

From the SHAP summary visualizations above, the following features were selected for each model:

- **2-month horizon**: `'household_deposits_roc2'`, `'household_deposits_diff2'`, `'household_deposits_std3'`
- **5-month horizon**: `'household_deposits_roc5'`, `'household_deposits_diff5'`
- **8-month horizon**: `'household_deposits_roc8'`, `'household_deposits_diff8'`
- **14-month horizon**: `'household_deposits_diff12'`, `'household_deposits_diff11'`, `'household_loans_roc4'`, `'household_loans_roc10'`

Feature selection was guided by the SHAP analysis — specifically, **features with consistently high average SHAP values across cross-validation folds and relatively low standard deviation** were preferred. This approach favours stability and avoids selecting features that may appear important in only a few folds due to noise.

For example, although `cpi_housing_household_diff12` had a high mean SHAP value for the 14-month horizon, its standard deviation was nearly twice as large, indicating unstable importance across folds. As such, it was excluded from the final feature set.


## Hyperparameter Tuning

To optimize the performance of the residual Gradient Boosting model in the forecasting pipeline, a hyperparameter tuning process is performed using **Optuna**, with trial logging via **MLflow**. For each forecast horizon, the model is trained using **expanding window cross-validation**, where the final 12 months of the training set are held out for final testing evaluation. The objective function minimizes the **Root Mean Squared Error (RMSE)** of forecasts at the specified horizon.

The tuning process searches over key hyperparameters of the Gradient Boosting model and its rolling window configuration, including:
- Number of trees (`n_estimators`)
- Learning rate (`learning_rate`)
- Maximum tree depth (`max_depth`)
- Rolling window length (`window_length`)
- Subsampling rate (`subsample`)
- Minimum samples per leaf (`min_samples_leaf`)

Early stopping is applied to prevent excessive trials once performance stabilizes. After tuning, the best configuration is used to re-train the final model on the full training data, and performance metrics and forecasts are logged for further analysis.


In [0]:
import optuna
from optuna.integration.mlflow import MLflowCallback
import mlflow
import os

def forecast_pipeline(X, y, horizon, window_length = 120, sp = 12, n_estimators=200, learning_rate=0.05, max_depth=4, step_length = 1, initial_window = 120, subsample = 1.0, min_samples_leaf = 1):
    """
    Runs the full forecasting pipeline using AutoETS + Gradient Boosting.

    Parameters:
        X (pd.DataFrame): Feature matrix.
        y (pd.Series): Target time series.
        horizon (int): Forecast horizon (e.g., 2 months ahead).
        window_length (int): Number of past points used in GBT residual correction.
        sp (int): Seasonal period (set to 12 for the deposits data as strong yearly seasonaility observed).
        n_estimators (int): Number of trees for Gradient Boosting.
        learning_rate (float): Learning rate for Gradient Boosting.
        max_depth (int): Max depth of each tree.
        step_length (int): Step size between CV folds.
        initial_window (int): Initial training window for expanding CV.
        subsample (float): Fraction of samples to use per tree (for GB).
        min_samples_leaf (int): Minimum samples per leaf in GB trees.

    Returns:
        results (pd.DataFrame): Forecast vs actual results with timestamps.
        rmse (float): Root Mean Squared Error of predictions.
    """

    # Base ETS model
    base_forecaster = AutoETS(auto=True,sp = sp, n_jobs = -1)

    # Residual model (GBR) 
    regressor = GradientBoostingRegressor(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, subsample = subsample, min_samples_leaf = min_samples_leaf, random_state = 42)
    residual_forecaster = make_reduction(regressor, window_length=window_length, strategy="direct")
    residual_forecaster = TransformedTargetForecaster([
        ("scaler", MinMaxScaler()),
        ("regressor", residual_forecaster)
    ])

    # Combined residual boosting forecast model
    forecaster = ResidualBoostingForecaster(base_forecaster, residual_forecaster)

    # Expanding window cross-validation
    splitter = ExpandingWindowSplitter(
        initial_window=initial_window,
        step_length=step_length,
        fh=[horizon]
    )

    # Run evaluation
    cv_results = evaluate(
        forecaster=forecaster,
        y=y,
        X = X,
        cv=splitter,
        strategy="refit",
        return_data=True,   
    )

    # Collect predictions and true values
    results = pd.DataFrame(
        {'date':[s.index[0].to_timestamp() for s in cv_results["y_pred"]],
        'y_true' :[s.values[0] for s in cv_results["y_test"]],
        'y_pred' :[s.values[0] for s in cv_results["y_pred"]]})

    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(results.y_true, results.y_pred))
    return results, rmse

def run_forecast_with_lags_and_windows(df, h=2, n_folds=12, step_length = 10, window_length=120, sp=12, n_estimators=100, learning_rate=0.1, max_depth=3, subsample = 1, min_samples_leaf = 1):

    """Wrapper for running forecast pipeline with CV-friendly expanding window setup. See forecast_pipeline for parameter details."""

    X = df.drop(columns='household_deposits')
    y = df['household_deposits']


    # Compute initial window to get approximately n_folds with given step_length
    N = len(X)
    initial_window = N - ((n_folds - 1) * step_length + h)  # Back-calculate initial window so that last fold is recent

    results, rmse = forecast_pipeline(
            X, y,
            horizon=h,
            window_length=window_length,
            sp=sp,
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            subsample = subsample, 
            min_samples_leaf=min_samples_leaf,
            step_length = step_length,
            initial_window = initial_window
        )
    
    return {
            "rmse": rmse,
            "results": results
        }
    
# Optuna Tuning definition
def early_stopping_callback(patience=30):
    """Returns a callback that stops the study if no improvement is seen in 'patience' trials."""
    def callback(study, trial):
        if study.best_trial.number + patience <= trial.number:
            print(f"\nEarly stopping triggered: No improvement in the last {patience} trials.")
            study.stop()
    return callback

def objective(trial, df, h=2, n_folds=10, step_length=6):
    """
    Objective function for Optuna to minimize RMSE.

    Args:
        trial (optuna.trial): Optuna trial object.
        df (pd.DataFrame): Feature matrix + target.
        h (int): Forecast horizon.
        n_folds (int): Number of CV folds.
        step_length (int): Step size for CV.

    Returns:
        float: RMSE of the trial configuration.
    """

    # Hyperparameter search space
    n_estimators = trial.suggest_int("n_estimators", 100, 300)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.1)
    max_depth = trial.suggest_int("max_depth", 2,5)
    window_length = trial.suggest_int("window_length", 90, 200)
    subsample = trial.suggest_float("subsample", 0.7, 1.0)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 5)


    try:
        result = run_forecast_with_lags_and_windows(
            df=df,
            h=h,
            n_folds=n_folds,
            step_length=step_length,
            window_length=window_length,
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            subsample = subsample, 
            min_samples_leaf = min_samples_leaf
        )
    except Exception as e:
        trial.set_user_attr("error", str(e))
        return float("inf")

    rmse = result["rmse"]

    # Log trial parameters and metric manually to MLflow
    with mlflow.start_run(nested=True):
        mlflow.log_param("features", ",".join(df.columns))
        mlflow.log_params({
            "sp": 12,
            "n_estimators": n_estimators,
            "learning_rate": learning_rate,
            "max_depth": max_depth,
            "window_length": window_length,
            "subsample":subsample,
            "min_samples_leaf":min_samples_leaf, 
            "horizon": h
        })
        mlflow.log_metric("rmse", rmse)

    return rmse

def tune_with_optuna_mlflow(df,h=2,n_trials=50,n_folds=10,step_length=6):
    """
    Runs hyperparameter tuning using Optuna and logs results to MLflow.

    Args:
        df (pd.DataFrame): Dataset with features and target.
        h (int): Forecast horizon.
        n_trials (int): Max number of Optuna trials.
        n_folds (int): Number of CV folds.
        step_length (int): Step size for CV.

    Returns:
        Tuple[optuna.Study, dict]: The Optuna study and final results dictionary.
    """

    study = optuna.create_study(study_name=f"forecast_study_h{h}", direction="minimize", sampler = optuna.samplers.TPESampler(seed=42))


    # Optimize with early stopping
    study.optimize(
        lambda trial: objective(
            trial,
            df=df,
            h=h,
            n_folds=n_folds,
            step_length=step_length,
        ),
        n_trials=n_trials,
        callbacks=[early_stopping_callback()],
        show_progress_bar=False,
    )

    print("Best trial:")
    print(study.best_trial.params)
    print("Best RMSE:", study.best_value)

    # After optimization, re-run final model with best params
    best_params = study.best_trial.params

    best_result = run_forecast_with_lags_and_windows(
        df=df,
        h=h,
        n_folds=n_folds,
        step_length=step_length,
        window_length=best_params["window_length"],
        sp=12,
        n_estimators=best_params["n_estimators"],
        learning_rate=best_params["learning_rate"],
        max_depth=best_params["max_depth"],
        subsample=best_params["subsample"],
        min_samples_leaf=best_params["min_samples_leaf"]
    )

    # Log the final best model and results
    with mlflow.start_run(run_name=f"shap_final_model_h{h}", nested=True):
        mlflow.log_params(best_params)
        mlflow.log_metric("rmse_final", best_result["rmse"])

        results_df = best_result["results"]
        csv_path = f"tuning_results/Shap_optuna_final_results_h{h}.csv"
        results_df.to_csv(csv_path, index=False)
        mlflow.log_artifact(csv_path)

        df_trials = study.trials_dataframe()
        csv_trials_path = f"tuning_results/Shap_optuna_trial_history_h{h}.csv"
        df_trials.to_csv(csv_trials_path, index=False)
        mlflow.log_artifact(csv_trials_path)


    return study, best_result


**A note on AutoETS parameters**:  
AutoETS automatically selects the optimal combination of error, trend, and seasonality components based on the corrected Akaike Information Criterion (AICc), making it highly adaptive to changes in the underlying temporal structure of the data. Unlike manually specifying a fixed ETS configuration — which may be more appropriate for stable and predictable time series — AutoETS dynamically re-evaluates and adjusts its components (e.g., from additive to damped trends, or seasonal to non-seasonal) as new data becomes available. This adaptability is particularly valuable in real-world forecasting, where patterns can evolve over time due to economic shocks, policy interventions, or structural changes. As a result, AutoETS offers a more flexible and scalable solution for time series modeling in dynamic environments.

A key assumption in this pipeline is that AutoETS will always return the best-fitting ETS structure given the available data at each point in time — effectively capturing the components that are explainable by traditional time series decomposition. This allows the **residual Gradient Boosting model** to focus on learning any remaining nonlinear patterns or relationships driven by exogenous variables that AutoETS cannot account for.

Cross-validation further reinforces this setup: with 10 folds and a 6-month step length, the model evaluates a range of data windows — each potentially resulting in different AutoETS configurations. This variability exposes the boosting model to a diverse set of residual patterns, strengthening its ability to generalize and complement the base model.

The only manually specified parameter for AutoETS is the **seasonal periodicity (`sp`)**, which defines the recurring seasonal cycle. For monthly data, `sp=12` captures annual seasonality, while `sp=4` would be appropriate for quarterly data.  
Exploratory analysis (see below) confirms a strong and stable annual pattern in household deposits, justifying the choice of `sp=12`.



In [0]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Load and clean time series
y = raw_df['household_deposits']

# Decompose with different seasonal periods
periods = [1, 4, 6, 12]
decompositions = [seasonal_decompose(y, period=p) for p in periods]

# Plot seasonal components in 2x2 grid
fig, axes = plt.subplots(2, 2, figsize=(14, 8), sharex=True)
axes = axes.flatten()

for i, p in enumerate(periods):
    decompositions[i].seasonal.plot(ax=axes[i])
    axes[i].set_title(f"Seasonal Component (sp={p})")
    axes[i].set_ylabel("Seasonality")

plt.suptitle("Seasonal Decomposition of Household Deposits", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()



Observe clear seasonality trend for sp = 12 which is also expected for monthly data

**Hyperparameter Tuning for Residual Gradient Booster**

The code below performs hyperparameter tuning for the residual gradient booster as described in the earlier section. 

In [0]:
## Hyperparameter Tuning

# Initial setup based on SHAP importance results
best_features = {
    2:['household_deposits_roc2','household_deposits_diff2','household_deposits_std3'],
    5:['household_deposits_roc5', 'household_deposits_diff5'],
    8:['household_deposits_roc8','household_deposits_diff8'],
    14:['household_deposits_diff12', 'household_deposits_diff11', 'household_loans_roc4', 'household_loans_roc10']}

best_lags = {
    2:[2],
    5:[5],
    8:[8],
    14:[4,10,11,12]
}

best_rolling_windows = {
    2:[3],
    5:[],
    8:[],
    14:[]
}

horizons = [2,5,8,14]
df_train, df_test = temporal_train_test_split(raw_df, test_size=12)

# Run tuning
for h in horizons:
    df = df_train[['date', 'household_deposits','household_loans']].copy()
    predictor_cols = [x for x in df.columns if x not in ['date', 'household_deposits']]
    # Feature engineering with variable lags/windows
    df_feats = create_features(df, 'household_deposits', predictor_cols, lags=best_lags[h], rolling_windows=best_rolling_windows[h])
    df_feats = df_feats.set_index('date')
    df_feats = df_feats.asfreq('MS')
    df_feats.index = df_feats.index.to_period("M")

    with mlflow.start_run(run_name=f"Parameter_Tuning_h{h}"):
        study, best_result = tune_with_optuna_mlflow(
            df = df_feats[['household_deposits']+best_features[h]],
            h = h,
            n_trials = 100,       
            n_folds = 10,
            step_length = 6
        )
        



In [0]:
## Save the parameters 
# Combine all the results from the feature selection and hyperparameter tuning into a final parameter dictionary for each of the horizon forecast models

# Final parameter dictionary
param_dict = {'model_forecast_h14': {'learning_rate':0.09556428757689246, 'max_depth':4, 'min_samples_leaf':1, 'n_estimators':175, 'sp':12, 'subsample':0.7468055921327309, 'window_length':156, 'best_features':['household_deposits_diff12', 'household_deposits_diff11', 'household_loans_roc4', 'household_loans_roc10'], 'predictors':['household_loans'],'best_lags':[4,10,11,12], 'best_rolling_windows':[],'horizon':14},
 'model_forecast_h8': {'learning_rate':0.0323887571145074, 'max_depth':4, 'min_samples_leaf':1, 'n_estimators':277, 'sp':12, 'subsample':0.7629576088119175, 'window_length':150, 'best_features':['household_deposits_roc8','household_deposits_diff8'], 'predictors':[], 'best_lags':[8], 'best_rolling_windows':[],'horizon':8},
 'model_forecast_h5': {'learning_rate':0.08240638205278353, 'max_depth':3, 'min_samples_leaf':1, 'n_estimators':119, 'sp':12, 'subsample':0.9521611575184935, 'window_length':129, 'best_features':['household_deposits_roc5', 'household_deposits_diff5'], 'predictors':[], 'best_lags':[5], 'best_rolling_windows':[],'horizon':5},
 'model_forecast_h2': {'learning_rate':0.07344290290006429, 'max_depth':4, 'min_samples_leaf':3, 'n_estimators':200, 'sp':12, 'subsample':0.7643393094474292, 'window_length':160, 'best_features':['household_deposits_roc2','household_deposits_diff2','household_deposits_std3'], 'predictors':[], 'best_lags':[2], 'best_rolling_windows':[3],'horizon':2},
  'model_forecast_h1': {'learning_rate':0.07344290290006429, 'max_depth':4, 'min_samples_leaf':3, 'n_estimators':200, 'sp':12, 'subsample':0.7643393094474292, 'window_length':160, 'best_features':['household_deposits_roc2','household_deposits_diff2','household_deposits_std3'], 'predictors':[], 'best_lags':[2], 'best_rolling_windows':[3],'horizon':1}         
}

with open("model_configs.json", "w") as f:
    json.dump(param_dict, f, indent=4)

# Save separately for production
# Create a directory to store config files if it doesn't exist
config_dir = Path("../configs")
config_dir.mkdir(exist_ok=True)

# Save each config to a separate file
for model_name, params in param_dict.items():
    config_path = config_dir / f"{model_name}.json"
    params['model_name'] = model_name
    params['target'] = 'household_deposits'
    with open(config_path, "w") as f:
        json.dump(params, f, indent=4)
